## Exploring the Sem Eval 2018 dataset using Autolabel

#### Set up the API keys for the providers you want to use

In [ ]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

#### Install the autolabel library

In [ ]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [2]:
from autolabel import get_data

get_data('twitter_emotion_detection')

2023-06-27 17:26:07 autolabel.utils ERROR: twitter_emotion_detection not in list of available datasets: ['banking', 'civil_comments', 'ledgar', 'walmart_amazon', 'company', 'squad_v2', 'sciq', 'conll2003', 'movie_reviews']. Exiting...


This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

In [3]:
import json

from autolabel import LabelingAgent

In [4]:
# load the config
with open('config_twitter_emotion_detection.json', 'r') as f:
    config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `multilabel_classification` (since it's a multi label classification task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.labels`: `['neutral', 'anger', 'anticipation', 'disgust', 'fear', ...]` (the full list of labels to choose from)
* `prompt.task_guidelines`: `'You are an expert at classifying tweets as...` (how we describe the task to the LLM)
* `prompt.few_shot_num`: 5 (how many labeled examples to provide to the LLM)

In [5]:
config

{'task_name': 'EmotionClassification',
 'task_type': 'multilabel_classification',
 'dataset': {'label_column': 'labels', 'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.\nYour job is to correctly label the provided input example into one or more of the following categories:\n{labels}',
  'output_guidelines': 'You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"',
  'labels': ['neutral',
   'anger',
   'anticipation',
   'disgust',
   'fear',
   'joy',
   'love',
   'optimism',
   'pessimism',
   'sadness',
   'surprise',
   'trust'],
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',
  'few_shot_num': 5,
  'example_template': 'Input: {example}\nOutput: {label}'}}

In [6]:
# create an aggent for labeling
agent = LabelingAgent(config=config)

In [7]:
agent.plan('test.csv')

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $8.0006 │
│ Number of Examples       │ 3259    │
│ Average cost per example │ $0.0025 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.
Your job is to correctly label the provided input example into one or more of the following categories:
neutral
anger
anticipation
disgust
fear
joy
love
optimism
pessimism
sadness
surprise
trust

You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"

Some examples with their output answers are provided below:

Input: @MaryamNSharif I think just becz u have so much terror in pak nd urself being  a leader u forgot d difference btw a leader nd terrorist !
Output: 

Input: In wake of fresh #terror threat and sounding of alert in #Mumbai, praying for safety &amp; security of everybody in the city #Maharashtra #news
Output: 

Input: Somewhere I rd a rpt tht Pakis wr afraid of TSD &amp; asked it 2 shut dn. Congis obliged &amp; exposed it,hounded them.time to revive. #BadlaofUri
O

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [8]:
# now, do the actual labeling
labels, df, metrics_list = agent.run('test.csv', max_items=100)

Output()

Actual Cost: 0.0465


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.4769 │ 100     │ 0.07     │ 1.0             │
└────────┴─────────┴──────────┴─────────────────┘

Total number of failures: 0

We are at a 0.4769 f1 when labeling the first 100 examples. Let's see if we can use confidence scores to improve f1 further by removing the less confident examples from our labeled set.

### Compute confidence scores

In [9]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ['REFUEL_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

In [10]:
# set `compute_confidence` -> True
config["model"]["compute_confidence"] = True

In [11]:
agent = LabelingAgent(config=config)

In [12]:
agent.plan('test.csv')

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $8.0006 │
│ Number of Examples       │ 3259    │
│ Average cost per example │ $0.0025 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.
Your job is to correctly label the provided input example into one or more of the following categories:
neutral
anger
anticipation
disgust
fear
joy
love
optimism
pessimism
sadness
surprise
trust

You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"

Some examples with their output answers are provided below:

Input: @MaryamNSharif I think just becz u have so much terror in pak nd urself being  a leader u forgot d difference btw a leader nd terrorist !
Output: 

Input: In wake of fresh #terror threat and sounding of alert in #Mumbai, praying for safety &amp; security of everybody in the city #Maharashtra #news
Output: 

Input: Somewhere I rd a rpt tht Pakis wr afraid of TSD &amp; asked it 2 shut dn. Congis obliged &amp; exposed it,hounded them.time to revive. #BadlaofUri
O

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [13]:
labels, df, metrics_list = agent.run('test.csv', max_items=100)

Output()

2023-06-27 17:30:41 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-27 17:30:51 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-27 17:30:55 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-27 17:31:08 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


...

2023-06-27 17:33:02 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-27 17:33:05 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


Metric: auroc: 0.4808
Actual Cost: 0.0037


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃ threshold ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 0.4    │ 1       │ 0.0      │ 0.01            │ 0.8155    │
│ 0.5519 │ 23      │ 0.0      │ 0.23            │ 0.6878    │
│ 0.5877 │ 25      │ 0.08     │ 0.25            │ 0.6591    │
│ 0.5807 │ 28      │ 0.0714   │ 0.28            │ 0.6411    │
│ 0.5952 │ 29      │ 0.1034   │ 0.29            │ 0.632     │
│ 0.5711 │ 45      │ 0.0667   │ 0.45            │ 0.5251    │
│ 0.5804 │ 46      │ 0.087    │ 0.46            │ 0.524     │
│ 0.5597 │ 57      │ 0.0702   │ 0.57            │ 0.4657    │
│ 0.5673 │ 58      │ 0.0862   │ 0.58            │ 0.4649    │
│ 0.4872 │ 84      │ 0.0595   │ 0.84            │ 0.3474    │
│ 0.4932 │ 85      │ 0.0706   │ 0.85            │ 0.3409    │
│ 0.4724 │ 98      │ 0.0612   │ 0.98            │ 0.2126    │
│ 0.4777 │ 99      │ 0.0707   │ 0.99            │ 0.0635    │
│ 0.4729 │ 100     │ 0.07     │ 1.0             │ 0.0193    │
└────────┴─────────┴──────────┴─────────────────┴───────────┘

Total number of failures: 0

Looking at the table above, we can see that if we set the confidence threshold at `0.632`, we are able to label at 0.6 f1 and getting a completion rate of 29%. This means, we would ignore all the data points where confidence score is less than `0.632` (which would end up being around 70% of all samples). This would, however, guarantee a higher quality labeled dataset for us.